In [ ]:
import healpix as hp
import numpy as np
import xarray as xr
import intake
import numcodecs
import zarr

import easygems.healpix as egh
import easygems.remap as egr

import matplotlib.pyplot as plt

import dask.array as da

import glob
import pandas as pd

import cartopy.crs as ccrs
import cartopy.feature as cf

import warnings
warnings.filterwarnings("ignore", category=FutureWarning) # don't warn us about future package conflicts

In [ ]:
plt.rcParams['figure.figsize'] = [12, 8]

In [ ]:
current_location = "NCAR"
cat = intake.open_catalog("https://digital-earths-global-hackathon.github.io/catalog/catalog.yaml")[current_location]

In [ ]:
#list(cat)

In [ ]:
# Query a data set
#pd.DataFrame(cat["scream2D_hrly"].describe()["user_parameters"])


In [ ]:
ds = cat["scream2D_hrly"](zoom=6).to_dask() 
ds = ds.pipe(egh.attach_coords)
#ds

In [ ]:
mmdy=86400.*1000. #mm/dy
mmhr=3600.*1000. #mm/hr

### Precipitation Frequency
Start with a month... pull out the data and make a mask by a threshold? 

Or save a precip mask 

In [ ]:
threshold = 1  # example:  1 mm/dy

# Step 1: Create binary mask (1 if pr > threshold, else 0)
mask = (ds['pr']*mmdy > threshold)

# Step 2: Group by month and count
count = mask.groupby("time.month").sum("time")  # Number of hours above threshold

# Optional: frequency (fraction of time exceeding threshold)
frequency = mask.groupby("time.month").mean("time")  # Fraction of time above threshold

In [ ]:
mo=6
z=7
egh.healpix_show(frequency.sel(month=mo),cmap="BuPu")
plt.title("Frequency of Precipitation, Month="+str(mo)+", zoom "+str(z))

In [ ]:
del mask,count

In [ ]:
### High resolution

In [ ]:
z=10
dshi = cat["scream2D_hrly"](zoom=z).to_dask() 
dshi = dshi.pipe(egh.attach_coords)
#dshi

In [ ]:
# Step 1: Create binary mask (1 if pr > threshold, else 0)
mask = (dshi['pr']*mmdy > threshold)

# Step 2: Group by month and count
count = mask.groupby("time.month").sum("time")  # Number of hours above threshold

# Optional: frequency (fraction of time exceeding threshold)
frequency_hi = mask.groupby("time.month").mean("time")  # Fraction of time above threshold

In [ ]:
%%time
mo=6
projection = ccrs.Robinson()
fig, axs = plt.subplots(subplot_kw={"projection": projection})
axs.set_global()
mappable = egh.healpix_show(frequency_hi.sel(month=mo),ax=axs,cmap="BuPu")

#Add map features
axs.add_feature(cf.COASTLINE, linewidth=0.8)
#ax.add_feature(cf.BORDERS, linewidth=0.4)

#Add colorbar
fig.colorbar(
    mappable, ax=axs, orientation='vertical', shrink=0.7, pad=0.05,
    label="Precipitation Frequency (>1 mm)"
)
plt.title("SCREAM Frequency of Precipitation, zoom "+str(z)+", Month="+str(mo))

### Zonal Means

In [ ]:
%%time
zmlo = (
    frequency
    .groupby("lat")
    .mean()
).compute()

In [ ]:
zmhi= (
    frequency_hi
    .groupby("lat")
    .mean()
)

In [ ]:
%%time
mo=6
zmlo.isel(month=6).plot(label='Zoom 7')
zmhi.isel(month=6).plot(label='Zoom 10')
plt.legend()
plt.title("Zonal Mean Precip Frequency, mo="+str(mo))

### Next Steps
- Now make the same plot from ICON, eventually loop over models. Also have observations.
- Seasonal would be ideal, but some models only have a month (MPAS): Start with MPAS, then ICON
- OBservations (IMERGE) are zoom=9